# Agenda

1. Iterators and the iterator protocol
    - The protocol itself
    - Making our classes iterable
    - Generators and generator functions
    - Generator expressions / generator comprehensions
    - `itertools`
2. Decorators
3. Concurrency
    - Threads
    - Multiprocessing
    - `asyncio`
